This time  we ll do the same as bank project(1) but we ll try to handle the problem of unbalanced train set


In [1]:
import pandas as pd
import numpy as np 
import scipy.optimize as opt 
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
data = pd.read_excel('Dataset2Use_Assignment1.xlsx')
data.sort_values(by=["ΕΝΔΕΙΞΗ ΑΣΥΝΕΠΕΙΑΣ (=2) (ν+1)"],ascending=True ,inplace=True )
data = data.reset_index(drop=True)
print(data.info())
x= data.iloc[:,:-2]
y= data.iloc[:,-2]
#print(y.head())
y, levels = pd.factorize(y)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10716 entries, 0 to 10715
Data columns (total 13 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   365* ( Β.Υ / Κοστ.Πωλ )                         10716 non-null  float64
 1   Λειτ.Αποτ/Συν.Ενεργ. (ROA)                      10716 non-null  float64
 2   ΧΡΗΜ.ΔΑΠΑΝΕΣ / ΠΩΛΗΣΕΙΣ                         10716 non-null  float64
 3    ΠΡΑΓΜΑΤΙΚΗ ΡΕΥΣΤΟΤΗΤΑ :  (ΚΕ-ΑΠΟΘΕΜΑΤΑ) / Β.Υ  10716 non-null  float64
 4   (ΑΠΑΙΤ.*365) / ΠΩΛ.                             10716 non-null  float64
 5   Συν.Υποχρ/Συν.Ενεργ                             10716 non-null  float64
 6   Διάρκεια Παραμονής Αποθεμάτων                   10716 non-null  float64
 7   Λογαριθμος Προσωπικού                           10716 non-null  float64
 8   ΕΝΔΕΙΞΗ ΕΞΑΓΩΓΩΝ                                10716 non-null  int64  
 9   ΕΝΔΕΙΞΗ ΕΙΣΑΓΩΓΩΝ                      

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7,random_state=1)



In [4]:
count = pd.Series(y_train).value_counts()
print(count)
num_of_healthy_needed=count[1]*3
#count_test_y = pd.Series(y_test).value_counts()
#print(count_test_y)
#print(count[1]/(count[2]+count[1]))
#num_of_healthy_needed=count[1]*3
print('num_of_healthy_needed= ',num_of_healthy_needed) 

0    7326
1     175
dtype: int64
num_of_healthy_needed=  525


we delete random values from  train set so as to make the set balanced

In [5]:
x_train['y']= y_train
x_train= x_train.sort_values(by=['y'],ascending=True)
x_train=x_train.reset_index(drop= True)
import random 
random.seed(1)
delete =random.sample(range(count[0]), count[0]-num_of_healthy_needed)
x_train= x_train.drop(delete)
df_suffled= x_train.sample(frac=1, random_state=1).reset_index(drop=True)
y_train= df_suffled.iloc[:,-1]
x_train= df_suffled.iloc[:,:-1]

print(y_train)

0      1
1      1
2      0
3      1
4      0
      ..
695    0
696    1
697    0
698    0
699    0
Name: y, Length: 700, dtype: int64


we check again if the train set is balanced

In [6]:
count1 = pd.Series(y_train).value_counts()
print(count1)

count2 = pd.Series(y_test).value_counts()
print(count2)


0    525
1    175
Name: y, dtype: int64
0    3142
1      73
dtype: int64


# Min Max Scaling

In [7]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)


In [8]:
print(X_train)

[[4.36058256e-01 5.90937481e-01 3.53048480e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.62135933e-01 5.46721262e-01 4.58636460e-01 ... 0.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [4.81198462e-01 5.37653501e-01 1.29000000e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.31347858e-01 5.49308327e-01 3.79128265e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [1.45716746e-01 6.45570850e-01 8.21840150e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [8.93072606e-01 6.04690285e-01 2.45587720e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


# LogisticRegression

In [9]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train) #fit the model using the training data
#now check for both train and test data, how well the model learned the patterns
#y_pred_train = logreg.predict(X_train)
y_pred_test= logreg.predict(X_test)
y_pred_train= logreg.predict(X_train)
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
pre_train = precision_score(y_train, y_pred_train, average='macro')
pre_test = precision_score(y_test, y_pred_test, average='macro')
rec_train = recall_score(y_train, y_pred_train, average='macro')
rec_test = recall_score(y_test, y_pred_test, average='macro')
f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')
print('confusion_matrix= ','\n' ,+confusion_matrix(y_test,y_pred_test))
TN=(confusion_matrix(y_test,y_pred_test)[0,0])
print("TN=",TN)
FP= (confusion_matrix(y_test,y_pred_test)[0,1])
print('FP=',FP)
FN= (confusion_matrix(y_test,y_pred_test)[1,0])
print('FN=',FN)
TP= (confusion_matrix(y_test,y_pred_test)[1,1])
print('TP=',TP)
print('acc_test= ',acc_test)
print('acc_train= ',acc_train)
print('rec_test= ',rec_test)
print('rec_train= ',rec_train)
print('f1_test= ',f1_test)
print('f1_train= ',f1_train)
print('precision_train= ',pre_train)
print('precision_test= ',pre_test)

confusion_matrix=  
 [[3009  133]
 [  50   23]]
TN= 3009
FP= 133
FN= 50
TP= 23
acc_test=  0.9430793157076205
acc_train=  0.7971428571428572
rec_test=  0.6363693834308485
rec_train=  0.6419047619047619
f1_test=  0.5856809966557022
f1_train=  0.6626345728288464
precision_train=  0.7545839760988851
precision_test=  0.5655453432913387


# DecisionTreeClassifier

In [10]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth= 6)
clf.fit(X_train, y_train) #fit the model using the training data
#now check for both train and test data, how well the model learned the patterns
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
pre_train = precision_score(y_train, y_pred_train, average='macro')
pre_test = precision_score(y_test, y_pred_test, average='macro')
rec_train = recall_score(y_train, y_pred_train, average='macro')
rec_test = recall_score(y_test, y_pred_test, average='macro')
f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')
print(confusion_matrix(y_test,y_pred_test))
TN=(confusion_matrix(y_test,y_pred_test)[0,0])
print("TN=",TN)
FP= (confusion_matrix(y_test,y_pred_test)[0,1])
print('FP=',FP)
FN= (confusion_matrix(y_test,y_pred_test)[1,0])
print('FN=',FN)
TP= (confusion_matrix(y_test,y_pred_test)[1,1])
print('TP=',TP)
print('acc_test= ',acc_test)
print('acc_train= ',acc_train)
print('rec_test= ',rec_test)
print('rec_train= ',rec_train)
print('f1_test= ',f1_test)
print('f1_train= ',f1_train)
print('precision_train= ',pre_train)
print('precision_test= ',pre_test)


[[2754  388]
 [  39   34]]
TN= 2754
FP= 388
FN= 39
TP= 34
acc_test=  0.8671850699844479
acc_train=  0.9057142857142857
rec_test=  0.6711326002982133
rec_train=  0.8476190476190476
f1_test=  0.5327138274063294
f1_train=  0.8669032830523514
precision_train=  0.892878670455524
precision_test=  0.5333026201251266


# KNeighborsClassifier

In [11]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train) #fit the model using the training data
#now check for both train and test data, how well the model learned the patterns
y_pred_train = knn.predict(X_train)
y_pred_test = knn.predict(X_test)
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
pre_train = precision_score(y_train, y_pred_train, average='macro')
pre_test = precision_score(y_test, y_pred_test, average='macro')
rec_train = recall_score(y_train, y_pred_train, average='macro')
rec_test = recall_score(y_test, y_pred_test, average='macro')
f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')
print(confusion_matrix(y_test,y_pred_test))
TN=(confusion_matrix(y_test,y_pred_test)[0,0])
print("TN=",TN)
FP= (confusion_matrix(y_test,y_pred_test)[0,1])
print('FP=',FP)
FN= (confusion_matrix(y_test,y_pred_test)[1,0])
print('FN=',FN)
TP= (confusion_matrix(y_test,y_pred_test)[1,1])
print('TP=',TP)
print('acc_test= ',acc_test)
print('acc_train= ',acc_train)
print('rec_test= ',rec_test)
print('rec_train= ',rec_train)
print('f1_test= ',f1_test)
print('f1_train= ',f1_train)
print('precision_train= ',pre_train)
print('precision_test= ',pre_test)

[[2889  253]
 [  44   29]]
TN= 2889
FP= 253
FN= 44
TP= 29
acc_test=  0.9076205287713841
acc_train=  0.8328571428571429
rec_test=  0.6583691567189557
rec_train=  0.7228571428571429
f1_test=  0.5572456964006259
f1_train=  0.7474957684730431
precision_train=  0.7981390878909662
precision_test=  0.5439175873467246


# Linear Discriminant Analysis


In [12]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train) #fit the model using the training data
#now check for both train and test data, how well the model learned the patterns
y_pred_train = lda.predict(X_train)
y_pred_test = lda.predict(X_test)
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
pre_train = precision_score(y_train, y_pred_train, average='macro')
pre_test = precision_score(y_test, y_pred_test, average='macro')
rec_train = recall_score(y_train, y_pred_train, average='macro')
rec_test = recall_score(y_test, y_pred_test, average='macro')
f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')
print(confusion_matrix(y_test,y_pred_test))
TN=(confusion_matrix(y_test,y_pred_test)[0,0])
print("TN=",TN)
FP= (confusion_matrix(y_test,y_pred_test)[0,1])
print('FP=',FP)
FN= (confusion_matrix(y_test,y_pred_test)[1,0])
print('FN=',FN)
TP= (confusion_matrix(y_test,y_pred_test)[1,1])
print('TP=',TP)
print('acc_test= ',acc_test)
print('acc_train= ',acc_train)
print('rec_test= ',rec_test)
print('rec_train= ',rec_train)
print('f1_test= ',f1_test)
print('f1_train= ',f1_train)
print('precision_train= ',pre_train)
print('precision_test= ',pre_test)

[[2972  170]
 [  49   24]]
TN= 2972
FP= 170
FN= 49
TP= 24
acc_test=  0.9318818040435459
acc_train=  0.8042857142857143
rec_test=  0.637330729053129
rec_train=  0.66
f1_test=  0.5721203193395892
f1_train=  0.6829867343666842
precision_train=  0.7628003432494279
precision_test=  0.553745772718121


# NAIVE BAYES

In [13]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train) #fit the model using the training data
# now check for both train and test data, how well the model learned thepatterns
y_pred_train = gnb.predict(X_train)
y_pred_test = gnb.predict(X_test)
#print('attributes: ', svc.support_vectors_)
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
pre_train = precision_score(y_train, y_pred_train, average='macro')
pre_test = precision_score(y_test, y_pred_test, average='macro')
rec_train = recall_score(y_train, y_pred_train, average='macro')
rec_test = recall_score(y_test, y_pred_test, average='macro')
f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')
print(confusion_matrix(y_test,y_pred_test))
TN=(confusion_matrix(y_test,y_pred_test)[0,0])
print("TN=",TN)
FP= (confusion_matrix(y_test,y_pred_test)[0,1])
print('FP=',FP)
FN= (confusion_matrix(y_test,y_pred_test)[1,0])
print('FN=',FN)
TP= (confusion_matrix(y_test,y_pred_test)[1,1])
print('TP=',TP)
print('acc_test= ',acc_test)
print('acc_train= ',acc_train)
print('rec_test= ',rec_test)
print('rec_train= ',rec_train)
print('f1_test= ',f1_test)
print('f1_train= ',f1_train)
print('precision_train= ',pre_train)
print('precision_test= ',pre_test)

[[2774  368]
 [  33   40]]
TN= 2774
FP= 368
FN= 33
TP= 40
acc_test=  0.8752721617418352
acc_train=  0.7842857142857143
rec_test=  0.7154111768963142
rec_train=  0.6980952380952381
f1_test=  0.5494569397746403
f1_train=  0.703734846892299
precision_train=  0.7106481481481481
precision_test=  0.5431414461046264


## Support Vector Classifier

In [14]:
#import SVC and create an SVC object
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train) #fit the model using the training data
# now check for both train and test data, how well the model learned the patterns
y_pred_train = svc.predict(X_train)
y_pred_test = svc.predict(X_test)
#print('attributes: ', svc.support_vectors_)
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
pre_train = precision_score(y_train, y_pred_train, average='macro')
pre_test = precision_score(y_test, y_pred_test, average='macro')
rec_train = recall_score(y_train, y_pred_train, average='macro')
rec_test = recall_score(y_test, y_pred_test, average='macro')
f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')
print(confusion_matrix(y_test,y_pred_test))
TN=(confusion_matrix(y_test,y_pred_test)[0,0])
print("TN=",TN)
FP= (confusion_matrix(y_test,y_pred_test)[0,1])
print('FP=',FP)
FN= (confusion_matrix(y_test,y_pred_test)[1,0])
print('FN=',FN)
TP= (confusion_matrix(y_test,y_pred_test)[1,1])
print('TP=',TP)
print('acc_test= ',acc_test)
print('acc_train= ',acc_train)
print('rec_test= ',rec_test)
print('rec_train= ',rec_train)
print('f1_test= ',f1_test)
print('f1_train= ',f1_train)
print('precision_train= ',pre_train)
print('precision_test= ',pre_test)

[[2993  149]
 [  52   21]]
TN= 2993
FP= 149
FN= 52
TP= 21
acc_test=  0.9374805598755832
acc_train=  0.8342857142857143
rec_test=  0.6201246043441486
rec_train=  0.6895238095238095
f1_test=  0.5701760162187941
f1_train=  0.7227381993006994
precision_train=  0.847284548953908
precision_test=  0.5532261180334203


## NEURAL NETWORK

In [15]:
import keras
from keras.models import Sequential
from keras.layers import InputLayer
from keras import layers
from sklearn.datasets import make_blobs
from keras.optimizers import Adam
#finally, build a feed forward neural network using keras
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],)))
model.add(keras.layers.Dense(32,activation='relu'))
model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dense(2, activation='sigmoid'))
#display the architecture
#model.summary()
#compile model using accuracy to measure model performance
opt=Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
#train the model
model.fit(X_train, keras.utils.to_categorical(y_train), epochs=100,verbose=True)
y_pred_train = np.argmax(model.predict(X_train),axis=1)
y_pred_test = np.argmax(model.predict(X_test),axis=1)


acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
pre_train = precision_score(y_train, y_pred_train, average='macro')
pre_test = precision_score(y_test, y_pred_test, average='macro')
rec_train = recall_score(y_train, y_pred_train, average='macro')
rec_test = recall_score(y_test, y_pred_test, average='macro')
f1_train = f1_score(y_train, y_pred_train, average='macro')
f1_test = f1_score(y_test, y_pred_test, average='macro')
print(confusion_matrix(y_test,y_pred_test))
TN=(confusion_matrix(y_test,y_pred_test)[0,0])
print("TN=",TN)
FP= (confusion_matrix(y_test,y_pred_test)[0,1])
print('FP=',FP)
FN= (confusion_matrix(y_test,y_pred_test)[1,0])
print('FN=',FN)
TP= (confusion_matrix(y_test,y_pred_test)[1,1])
print('TP=',TP)
print('acc_test= ',acc_test)
print('acc_train= ',acc_train)
print('rec_test= ',rec_test)
print('rec_train= ',rec_train)
print('f1_test= ',f1_test)
print('f1_train= ',f1_train)
print('precision_train= ',pre_train)
print('precision_test= ',pre_test)

Epoch 1/100
22/22 [==============================] - 0s 2ms/step - loss: 0.5933 - accuracy: 0.7100
Epoch 2/100
22/22 [==============================] - 0s 2ms/step - loss: 0.5034 - accuracy: 0.7500
Epoch 3/100
22/22 [==============================] - 0s 1ms/step - loss: 0.4678 - accuracy: 0.7643
Epoch 4/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4416 - accuracy: 0.7800
Epoch 5/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4284 - accuracy: 0.7814
Epoch 6/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4196 - accuracy: 0.7900
Epoch 7/100
22/22 [==============================] - 0s 1ms/step - loss: 0.4133 - accuracy: 0.8014
Epoch 8/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4202 - accuracy: 0.7986
Epoch 9/100
22/22 [==============================] - 0s 2ms/step - loss: 0.4138 - accuracy: 0.8071
Epoch 10/100
22/22 [==============================] - 0s 1ms/step - loss: 0.4022 - accuracy: 0.8129
Epoch 11/

22/22 [==============================] - 0s 1ms/step - loss: 0.2266 - accuracy: 0.8986
Epoch 84/100
22/22 [==============================] - 0s 1ms/step - loss: 0.2025 - accuracy: 0.9200
Epoch 85/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2095 - accuracy: 0.9100
Epoch 86/100
22/22 [==============================] - 0s 1ms/step - loss: 0.1981 - accuracy: 0.9186
Epoch 87/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1965 - accuracy: 0.9314
Epoch 88/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2169 - accuracy: 0.9043
Epoch 89/100
22/22 [==============================] - 0s 2ms/step - loss: 0.2110 - accuracy: 0.9129
Epoch 90/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1970 - accuracy: 0.9200
Epoch 91/100
22/22 [==============================] - 0s 1ms/step - loss: 0.1894 - accuracy: 0.9214
Epoch 92/100
22/22 [==============================] - 0s 2ms/step - loss: 0.1899 - accuracy: 0.9271
Epoch 93/100


In [16]:
model.save('C:/Users/Chrispdl/Desktop/ml and natural language processing/εργασία 1/1η άσκηση, shallow learning, classification/nnscprit')

INFO:tensorflow:Assets written to: C:/Users/Chrispdl/Desktop/ml and natural language processing/εργασία 1/1η άσκηση, shallow learning, classification/nnscprit\assets
